In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mm_functions as mmf
import params as pr
from scipy import interpolate

In [ ]:
# Data location
# prefix_dir = '/scratch2/raphaelr/processed_data/'
prefix_dir = '/Users/raphael/Documents/work/data/'

# Vectors for plotting
lon_qvec = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.eff_factor+1)
lat_qvec = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.eff_factor+1)
# Vectors for plotting HR
lon_qvec_hr = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.res_factor+1)
lat_qvec_hr = np.linspace(-pr.half_width,pr.half_width,pr.half_width*pr.res_factor+1)

In [ ]:
# Updated stuff below this line

In [ ]:
i_fig == 1

if i_fig == 1:
    # Cluster considered in plotting
    cluster_vec = np.arange(1,9+1)
    # List of variables and composites to produce
    var_list = ['precip_mask','swh']
    cluster_vec = [2,4,8]
    title_list = ['Rain','Wave']
    mm_list = [[0],[1],[0,1]]
    mm_ref_list = [[0],[1],[0,1]] # Reference list for computation of the statistical significance
    mm_ref_sig_list = [0,1,2]
    # List of plotting specifications
    colors_list = ['Blues','Reds','Purples']
    color_range_list = [np.linspace(0.02,0.20,10)*100,np.linspace(0.24,0.72,7)*100,np.linspace(0.015,0.15,10)*100] # 2m
    color_range_list = [np.linspace(0.04,0.24,11)*100,np.linspace(0.02,0.16,8)*100,np.linspace(0.006,0.06,10)*100] # 4m
    xlim_list = [-10,10]
    ylim_list = [-10,10]
    
elif i_fig == 2:
    # Cluster considered in plotting
    cluster_vec = np.arange(1,9+1)
    cluster_vec = [2,4,1]
    # List of variables and composites to produce
    var_list = ['precip','wind_mag']
    title_list = ['Rain','Wind']
    mm_list = [[0],[1],[0,1]]
    mm_ref_list = [[0],[1],[0,1]] # Reference list for computation of the statistical significance
    mm_ref_sig_list = [0,1,2]
    # List of plotting specifications
    colors_list = ['Blues','Reds','Purples']
    color_range_list = [np.linspace(0.04,0.24,11)*100,np.linspace(0.03,0.30,10)*100,np.linspace(0.004,0.044,11)*100]
    # color_range_list = [np.linspace(0.02,0.2,11)*100,np.linspace(0.03,0.30,10)*100,np.linspace(0.002,0.04,11)*100]
    xlim_list = [-10,10]
    ylim_list = [-10,10]
    
elif i_fig == 3:
    # Cluster considered in plotting
    cluster_vec = np.arange(1,9+1)
    cluster_vec = [6,7,5]
    # List of variables and composites to produce
    var_list = ['PM10','t2m']
    title_list = ['PM10','Heat']
    mm_list = [[0],[1],[0,1]]
    mm_ref_list = [[0],[1],[0,1]]# Reference list for computation of the statistical significance
    mm_ref_sig_list = [0,1,2]
    # List of plotting specifications
    colors_list = ['Blues','Reds','Purples']
    color_range_list = [np.linspace(0.04,0.20,9),np.linspace(0.05,0.5,10),np.linspace(0.02,0.10,9)]
    color_range_list = [np.linspace(0.02,0.10,9),np.linspace(0.05,0.5,10),np.linspace(0.01,0.05,9)]
    color_range_list = [np.linspace(0.06,0.30,9)*100,np.linspace(0.05,0.5,10)*100,np.linspace(0.04,0.20,9)*100]
    color_range_list = [np.linspace(0.06,0.36,6)*100,np.linspace(0.1,0.7,11)*100,np.linspace(0.06,0.24,10)*100]
    xlim_list = [-15,15]
    ylim_list = [-15,15]

In [ ]:
comp_sig = 1 # Compute significance (if 0, it will Load instead)
isig = 1 # Plot sgnificance
hide_sig = 0 # Hide insignificant area (usefull for noisy masked composites, like waves)
itopo = 0 # Plot topography

# significance map save directory
# sigpref = '/scratch2/raphaelr/processed_data/significance_maps/'
sigpref = '/Users/raphael/Documents/work/data/significance_maps/'

# Plotting
n_clus = len(cluster_vec)
n_mm = len(mm_list)
fig, axs = plt.subplots(n_clus,n_mm)
for ii, cluster_number in enumerate(cluster_vec):
    
    # Make MM composite for plotting
    mm_comp_list_plot, dens_list_plot  = mmf.make_composite(var_list,mm_list,prefix_dir,cluster_number,'plot')
    
    if comp_sig == 1:
        # Make MM composite for significance testing
        mm_comp_list, dens_list  = mmf.make_composite(var_list,mm_list,prefix_dir,cluster_number,'sig')
       # Make MM - MC composite
        # Note ref. values for statistical significance must exclude dynamical features from background computations
        %time mm_ref_comp_list = mmf.make_mc_composite(var_list,mm_ref_list,dens_list,prefix_dir,cluster_number)
        # This step to avoid recomputing duplicates
        mm_mc_comp_list = list(map(mm_ref_comp_list.__getitem__, mm_ref_sig_list)) 
        # Evaluate statistical significance
        mm_comp_sig_list = mmf.compute_sig(mm_comp_list,mm_mc_comp_list)
    
    for jj, mm_comp in enumerate(mm_comp_list_plot):
        if (n_clus>1) and (n_mm>1): ax=axs[ii,jj]
        elif (n_clus==1) and (n_mm>1): ax=axs[jj]
        elif (n_clus>1) and (n_mm==1): ax=axs[ii]
        else: ax=axs
        
        if comp_sig == 1:
            # Computing
            # Smooth the rendition of the confidence region
            f = interpolate.interp2d(lon_qvec, lat_qvec, mm_comp_sig_list[jj], kind='linear')
            conf_plot = f(lon_qvec_hr, lon_qvec_hr)
            if isig == 1: ax.contour(lon_qvec_hr,lat_qvec_hr,conf_plot,[0.25],colors='k',linewidths=0.5)
            if hide_sig == 1: mm_comp[conf_plot<0.25]=0
        else:
            # Loading
            analysis_name = "-".join(list(map(var_list.__getitem__, mm_list[jj])))
            sigdir= sigpref+'C'+str(cluster_number)+'-'+analysis_name+'.npy'
            conf_plot = np.load(sigdir)      
            if isig == 1: ax.contour(lon_qvec_hr,lat_qvec_hr,conf_plot,[1],colors='k',linewidths=0.5)
            if hide_sig == 1: mm_comp[conf_plot]=0  
            
        hcont = ax.contourf(lon_qvec_hr,lat_qvec_hr,mm_comp*100,color_range_list[jj],cmap=colors_list[jj],extend='max')
        
        fig.colorbar(hcont, ax=ax)
        ax.scatter(0,0,s=1.5,color='k')
        if itopo == 1: 
            elev_plot = np.load('/Users/raphael/Documents/work/data/elevation/C'+str(cluster_number)+'_elev.npy')
            ax.contour(lon_qvec_hr,lat_qvec_hr,elev_plot,[300],colors='brown',linewidths=0.6)
            grad = np.gradient(elev_plot,2/pr.res_factor)
#             grad_mag = np.sqrt(grad[0]**2 + grad[1]**2)
            ax.contour(lon_qvec_hr,lat_qvec_hr,grad[0],[100],colors='red',linewidths=0.7)

        
        # Setup axes
        ax.set_xlim(xlim_list) 
        ax.set_ylim(ylim_list)  
        if jj == 0: ax.set_ylabel('Latitude')
        if ii == n_clus-1: ax.set_xlabel('Longitude')
        analysis_name = "-".join(list(map(title_list.__getitem__, mm_list[jj])))
        ax.set_title('Clus. '+str(cluster_number)+' '+analysis_name)
        
        if comp_sig == 1:
            # Significance field
            sigvar = (conf_plot<0.25)
            analysis_name = "-".join(list(map(var_list.__getitem__, mm_list[jj])))
            sigdir= sigpref+'C'+str(cluster_number)+'-'+analysis_name
            np.save(sigdir, sigvar)

plt.subplots_adjust(hspace=0.25)    
fig.set_size_inches(4.1*n_mm,3.3*n_clus)

fig.savefig('mm_'+title_list[0]+'_'+title_list[1]+'_review1.pdf')
fig.savefig('dynamical_features.pdf')